In [53]:
#import packages
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from fbprophet import Prophet
import matplotlib.pyplot as plt
import os
import seaborn as sns

# read csv
def read_csv(filepath):

    return pd.read_csv(filepath) 

# look for headers
def csv_validation(df):
    df.columns = map(lambda x: str(x).upper(), df.columns)
    if not 'DATE' in df.columns:
        raise ValueError("Date column is missing")
        
    if not 'CLOSE' in df.columns:
        raise ValueError('Close column is missing')
        
    if df.empty:
        raise ValueError('csv is empty!')
    if df.shape[0] < 100:
        raise ValueError('Minimum 100 rows are required')

    return df


# drop null rows
def drop_null_rows(df):
    df.columns = df.columns.str.upper()
    if df['CLOSE'].isnull().values.any():
        df.dropna(axis=0, subset=['CLOSE'])
    if df['DATE'].isnull().values.any():
        df.dropna(axis=0, subset=['DATE'])
    return df

# feature extraction
def extract_date_and_close(df):
    df.columns = df.columns.str.upper()
    return df[["DATE","CLOSE"]]

# rename columns
def rename_columns(df):
   
    
   
    return df.rename(columns={'DATE':'ds','CLOSE':'y' })

def get_model(df):
    # Model initialization. Create an object of class Prophet.
    model = Prophet(daily_seasonality=True,yearly_seasonality=True)
    # model = Prophet(daily_seasonality=True)
    
    # Fit the data(train the model)
    model.fit(df)

    return model


def future_prediction(model,periods=3650):
    
    # Create a future data frame of future dates. Here 3650 is approximate number of days in 10 yrs time frame.
    future = model.make_future_dataframe(periods=periods)

    # Prediction for future dates.
    forecast = model.predict(future)

    return forecast

def preprocess_forecast(forecast):
    forecast_valid = forecast[['ds','yhat']][:]
    return forecast_valid.rename(columns={'yhat': 'y'})


def main(filepath,periods=3650,future_analysis_csv = False,filename=""):
    
    try:
        if future_analysis_csv:
            if filename == "":
                raise ValueError("Please Provide output csv file name")
            
        df = read_csv(filepath)
        
        df_validate = csv_validation(df)
        df_no_null = drop_null_rows(df)
       
        df_feature_extraction = extract_date_and_close(df_no_null)
     
        df_rename_columns = rename_columns(df_feature_extraction)
       
        model = get_model(df_rename_columns)
       

        forecast = future_prediction(model,periods)
        if future_analysis_csv:
            forecast.to_csv(f'{filename}.csv')
        
        return preprocess_forecast(forecast) 
    
    except Exception as e:
        
        raise ValueError(e)














In [54]:
import plotly.graph_objects as go
import plotly.express as px
def preprocess_forecast_rename(stock_name,forecast_valid):
    y_name = f"{stock_name} Close price"
    x_name = "Year"
    df = forecast_valid 
    mapping = {df.columns[0]:x_name,df.columns[1]:y_name}
    return df.rename(columns=mapping),x_name,y_name

class StockMarket:
    def __init__(self, filepath,stock_name,future_analysis_csv=False,filename=""):
        if future_analysis_csv:
            if filename == "":
                raise ValueError("Please Provide output csv file name")
            self.forecast_valid = preprocess_forecast_rename(stock_name=stock_name,forecast_valid=main(filepath,future_analysis_csv=True,filename=filename))
            self.stock_name = stock_name
        else:
            self.forecast_valid = preprocess_forecast_rename(stock_name=stock_name,forecast_valid=main(filepath))
            self.stock_name = stock_name
        
 

    def linechart(self):
       
       
        df,x_name,y_name = self.forecast_valid 
        title = f'{y_name} over time'

        print(df.tail(1))
      
      
        fig = go.Figure()
        # Create and style traces

        fig.add_trace(go.Scatter(x=df[x_name], y=df[y_name], name=title,line=dict(color='firebrick', width=8,dash='dash'))) # dash options include 'dash', 'dot', and 'dashdot'))


        # Edit the layout
        fig.update_layout(title=title, xaxis_title=x_name, yaxis_title=y_name)


        fig.show()

    def histogram(self):
        
        df,x_name,y_name = self.forecast_valid
        fig = px.histogram(df, x=x_name,y=y_name, histnorm='probability density')
        fig.show()
        
        
        

In [55]:
filepath = 'bse_sensex_50.csv'

In [56]:
# main(filepath)
predictions = StockMarket(filepath,"BSE SENSEX 50",future_analysis_csv=True,filename="bse_sensex_future")


In [57]:
predictions.linechart()

           Year  BSE SENSEX 50 Close price
6119 2030-12-30               52725.115731


In [58]:
predictions.histogram()